In [1]:
import pandas as pd

In [40]:
bores = pd.read_csv('NGIS_Bore.csv')

In [41]:
# take useful columns
bores = bores[[
    'HydroID',
    'HydroCode',
    'StateTerritory',
    'BoreDepth',
    'DrilledDate',
    'Latitude',
    'Longitude',
    'LandElev',
    'FTypeClass' # commerical, industrial, domestic etc..
]]

In [42]:
bores = bores.rename(columns={
    "HydroCode": "bore_id",
    'HydroID': 'hydro_id',
    'StateTerritory': 'state',
    'BoreDepth': 'bore_depth',
    'DrilledDate': 'drilled_date',
    'Latitude': 'lat',
    'Longitude': 'long',
    'LandElev': 'land_elevation',
    'FTypeClass': 'type_of_use'
})bores.set_index(['bore_id', 'state', 'hydro_id'], inplace=True, verify_integrity=True)
bores.set_index(['bore_id', 'state', 'hydro_id'], inplace=True, verify_integrity=True)

In [108]:
nsw_salinity = pd.read_csv('NSW/salinity_NSW.csv')
nsw_salinity = nsw_salinity[['hydroid', 'bore_id', 'bore_date', 'uom', 'result']]
nsw_salinity = nsw_salinity.rename(columns={
    'hydroid': 'hydro_id',
    'bore_date': 'last_salinity_measurement'
})
# only take latest salinity measurements
nsw_salinity = nsw_salinity.sort_values('last_salinity_measurement', ascending=False).drop_duplicates(['hydro_id', 'bore_id'])
nsw_salinity.set_index(['bore_id', 'hydro_id'], inplace=True, verify_integrity=True)


In [109]:
nsw_salinity.head()

,,last_salinity_measurement,uom,result
bore_id,hydro_id,,,
GW080073.1.1,10097525,2019-06-26 00:00:00,uS/cm,554.0
GW075409.2.2,10159509,2019-06-26 00:00:00,uS/cm,220.0
GW080838.1.1,10102662,2019-06-26 00:00:00,uS/cm,8343.0
GW080843.1.1,10104801,2019-06-26 00:00:00,uS/cm,3366.0
GW075411.1.1,10159605,2019-06-26 00:00:00,uS/cm,235.0


In [110]:
#nsw_salinity.join(other=bore_locations, how='left', on='bore_id', rsuffix='_r')

# nsw_salinity.bore_id = nsw_salinity.bore_id.str.strip()
# bore_locations.bore_id = bore_locations.bore_id.str.strip()

df = nsw_salinity.merge(bores, on=['bore_id', 'hydro_id'], how='left')



In [112]:
nsw_levels = pd.read_csv('NSW/level_NSW.csv') 
# SWL = standing water level
# RSWL = reduced standing water level

In [116]:
nsw_levels = nsw_levels[['hydroid', 'bore_id', 'bore_date', 'obs_point_datum', 'result']] # maybe take quality flag too?

In [119]:
nsw_levels

,hydroid,bore_id,bore_date,obs_point_datum,result
0,10002276,GW000001.1.1,1966-07-09 00:00:00,SWL,24.69
1,10002276,GW000001.1.1,1966-08-17 00:00:00,SWL,25.45
2,10002276,GW000001.1.1,1967-02-28 00:00:00,SWL,25.63
3,10002276,GW000001.1.1,1967-08-15 00:00:00,SWL,25.69
4,10002276,GW000001.1.1,1967-11-22 00:00:00,SWL,25.76
...,...,...,...,...,...
2381637,10161489,GW971623.3.3,2018-10-02 00:00:00,SWL,23.75
2381638,10161489,GW971623.3.3,2018-10-01 00:00:00,SWL,23.75
2381639,10161489,GW971623.3.3,2019-02-28 00:00:00,SWL,23.87
2381640,10161489,GW971623.3.3,2019-02-27 00:00:00,SWL,23.85


In [120]:
nsw_levels = nsw_levels.rename(columns={
    'hydroid': 'hydro_id',
    'bore_date': 'last_level_measurement'
})

In [121]:
nsw_levels = nsw_levels.sort_values('last_level_measurement', ascending=False).drop_duplicates(['hydro_id', 'bore_id'])


In [123]:
df2 = df.merge(nsw_levels, on=['bore_id', 'hydro_id'], how='left')

In [133]:
df2.set_index(['bore_id', 'hydro_id'], inplace=True, verify_integrity=True)

In [134]:
df2

,,last_salinity_measurement,uom,result_x,bore_depth,drilled_date,lat,long,land_elevation,type_of_use,last_level_measurement,obs_point_datum,result_y
bore_id,hydro_id,,,,,,,,,,,,
GW080073.1.1,10097525,2019-06-26 00:00:00,uS/cm,554.0,18.0,2000/06/09 00:00:00,-32.036200,150.855300,207.346,Monitoring,2019-09-01 00:00:00,SWL,10.25
GW075409.2.2,10159509,2019-06-26 00:00:00,uS/cm,220.0,100.0,2011/05/31 00:00:00,-34.230397,150.543889,307.360,Monitoring,2019-08-27 00:00:00,SWL,16.73
GW080838.1.1,10102662,2019-06-26 00:00:00,uS/cm,8343.0,NaN,NaN,-32.081299,151.984711,111.048,Unknown,2019-09-01 00:00:00,SWL,4.33
GW080843.1.1,10104801,2019-06-26 00:00:00,uS/cm,3366.0,NaN,NaN,-32.042597,151.967783,100.713,Unknown,2019-09-01 00:00:00,SWL,6.38
GW075411.1.1,10159605,2019-06-26 00:00:00,uS/cm,235.0,27.5,2011/06/22 00:00:00,-34.217500,150.549167,306.990,Monitoring,2019-09-01 00:00:00,SWL,13.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GW045546.1.1,10051156,1916-01-01 00:00:00,TDS,1083.0,123.4,1916/01/01 00:00:00,-32.071489,147.839433,NaN,Water Supply,NaN,NaN,NaN
GW005290.1.1,10002816,1914-01-01 00:00:00,TDS,1112.0,73.2,1914/01/01 00:00:00,-31.956850,147.786369,NaN,Stock and Domestic,NaN,NaN,NaN
GW004208.1.1,10031982,1908-01-01 00:00:00,TDS,665.0,250.6,1908/01/01 00:00:00,-30.336779,147.286181,126.200,Stock and Domestic,NaN,NaN,NaN


In [136]:
df2_poor_tds = df2[(df2.uom == 'TDS') & (1000 < df2.result_x) & (df2.result_x < 2000)]

In [138]:
df2_good_tds = df2[(df2.uom == 'TDS') & (df2.result_x < 1000)]

In [142]:
df2[df2.uom == 'uS/cm']

,,last_salinity_measurement,uom,result_x,bore_depth,drilled_date,lat,long,land_elevation,type_of_use,last_level_measurement,obs_point_datum,result_y
bore_id,hydro_id,,,,,,,,,,,,
GW080073.1.1,10097525,2019-06-26 00:00:00,uS/cm,554.0,18.0,2000/06/09 00:00:00,-32.036200,150.855300,207.346,Monitoring,2019-09-01 00:00:00,SWL,10.25
GW075409.2.2,10159509,2019-06-26 00:00:00,uS/cm,220.0,100.0,2011/05/31 00:00:00,-34.230397,150.543889,307.360,Monitoring,2019-08-27 00:00:00,SWL,16.73
GW080838.1.1,10102662,2019-06-26 00:00:00,uS/cm,8343.0,NaN,NaN,-32.081299,151.984711,111.048,Unknown,2019-09-01 00:00:00,SWL,4.33
GW080843.1.1,10104801,2019-06-26 00:00:00,uS/cm,3366.0,NaN,NaN,-32.042597,151.967783,100.713,Unknown,2019-09-01 00:00:00,SWL,6.38
GW075411.1.1,10159605,2019-06-26 00:00:00,uS/cm,235.0,27.5,2011/06/22 00:00:00,-34.217500,150.549167,306.990,Monitoring,2019-09-01 00:00:00,SWL,13.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GW028956.1.1,10020689,1938-01-01 12:00:00,uS/cm,513.0,115.8,1938/02/01 00:00:00,-29.800367,151.104722,NaN,Water Supply,NaN,NaN,NaN
GW000124.1.1,10004794,1933-01-01 12:00:00,uS/cm,573.0,62.8,1917/08/01 00:00:00,-35.537633,146.178482,NaN,Unknown,NaN,NaN,NaN
GW000145.1.1,10017286,1923-01-01 12:00:00,uS/cm,681.0,66.8,1917/10/01 00:00:00,-31.667600,149.205052,NaN,Water Supply,NaN,NaN,NaN


In [12]:
df3_poor_tds_locations = df3_poor_tds[['bore_id','Longitude', 'Latitude']]

In [13]:
df3_poor_tds_locations

,bore_id,Longitude,Latitude
68,GW000467.1.1,147.671233,-32.185184
78,GW000564.1.1,148.062105,-32.198409
337,GW003926.1.1,147.780913,-31.921220
1791,GW005289.1.1,147.786360,-31.955939
1792,GW005290.1.1,147.786369,-31.956850
...,...,...,...
151015,GW970210.1.1,150.307004,-31.236889
151019,GW970215.1.1,150.326619,-31.285664
151020,GW970216.1.1,150.326744,-31.285711
151063,GW970609.1.1,150.417851,-29.701731


In [14]:
import geojson

In [17]:
features = []
insert_features = lambda X: features.append(
        geojson.Feature(geometry=geojson.Point((X["Longitude"],
                                                X["Latitude"])),
                        properties=dict(id=X["bore_id"], icon="drinking-water-15")))
df3_poor_tds_locations.apply(insert_features, axis=1)

with open('map1.geojson', 'w', encoding='utf8') as fp:
    geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)


In [72]:
df3.to_csv('output.csv')

In [56]:
hydrochem_NSW = pd.read_csv('NSW/hydrochem_results_NSW.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
hydrochem_variables = hydrochem_NSW.variable.unique()

In [67]:
hydrochem_variables

array(['Temperature_Field', 'Ph_Field', 'Conductivity_Field', 'Ph',
       'Redox_Potential_Raw_Field', 'Oxygen_Dissolved_Field',
       'Electrical_Conductivity', 'Total_Dissolved_Solids', 'Calcium',
       'Magnesium', 'Sodium', 'Potassium', 'Bromine', 'Chloride',
       'Sulfate_As_S', 'Alk_Total_Caco3', 'Alk_Carbonate_Caco3',
       'Alk_Bicarbonate_Caco3', 'Aluminium', 'Iron', 'Manganese',
       'Copper', 'Zinc', 'Strontium', 'Lithium', 'Lead', 'Uranium',
       'Vanadium', 'Barium', 'Sulphur', 'Silicon_As_Sio2', 'Boron',
       'Fluoride', 'Iodine', 'Nitrate_As_N', 'Phosphate_As_P',
       'Oxygen_Gas', 'Nitrogen_Gas', 'Carbon_Dioxide_Gas', 'Methane_Gas',
       'Ethane_Gas', 'Propane_Gas', 'Iso_Butane_Gas', 'N_Butane_Gas',
       'Iso_Pentane_Gas', 'N_Pentane_Gas', 'Hexane_And_Higher_Gas',
       'Oxygen_18', 'C_13', 'C_14_Pmc', 'C_14_Pmc_Uncertainty',
       'Chlorine_36_Ratio', 'Chlorine_36_Uncertainty', 'Radon_222',
       'Helium_Gas', 'Hydrogen_Gas', 'Tritium', 'Deuterium'

In [69]:
for var in hydrochem_variables:
    df = hydrochem_NSW[hydrochem_NSW.variable == var]
    name = f'hydrochem_{var}_NSW.csv'.replace('/','_')
    df.to_csv('csvs/' + name)
    print(f'Processing {name}')

Processing hydrochem_Temperature_Field_NSW.csv
Processing hydrochem_Ph_Field_NSW.csv
Processing hydrochem_Conductivity_Field_NSW.csv
Processing hydrochem_Ph_NSW.csv
Processing hydrochem_Redox_Potential_Raw_Field_NSW.csv
Processing hydrochem_Oxygen_Dissolved_Field_NSW.csv
Processing hydrochem_Electrical_Conductivity_NSW.csv
Processing hydrochem_Total_Dissolved_Solids_NSW.csv
Processing hydrochem_Calcium_NSW.csv
Processing hydrochem_Magnesium_NSW.csv
Processing hydrochem_Sodium_NSW.csv
Processing hydrochem_Potassium_NSW.csv
Processing hydrochem_Bromine_NSW.csv
Processing hydrochem_Chloride_NSW.csv
Processing hydrochem_Sulfate_As_S_NSW.csv
Processing hydrochem_Alk_Total_Caco3_NSW.csv
Processing hydrochem_Alk_Carbonate_Caco3_NSW.csv
Processing hydrochem_Alk_Bicarbonate_Caco3_NSW.csv
Processing hydrochem_Aluminium_NSW.csv
Processing hydrochem_Iron_NSW.csv
Processing hydrochem_Manganese_NSW.csv
Processing hydrochem_Copper_NSW.csv
Processing hydrochem_Zinc_NSW.csv
Processing hydrochem_Stronti